In [1]:
######################## Part one ################################## 
from bs4 import BeautifulSoup
import requests
import pandas as pd 

In [2]:
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(URL)
soup = BeautifulSoup(response.text, 'html.parser')

table = soup.find('table', {'class':'wikitable sortable'}).tbody  # Obtained from inspect (wiki - right click)

In [3]:
rows = table.find_all('tr')
columns = [v.text.replace('\n', '') for v in rows[0].find_all('th')]

In [4]:
df = pd.DataFrame(columns = columns)

In [5]:
for i in range(1, len(rows)):
    tds = rows[i].find_all('td')

    if len(tds) == 3:
        values = [tds[0].text, tds[1].text, tds[2].text.replace('\n', '')]
    else:
        values = [td.text for td in tds]
    df = df.append(pd.Series(values, index=columns), ignore_index = True)

In [6]:
# Getting the names of the indexes for which the Borough has a value - Not assigned
indexNames = df[df['Borough'] == 'Not assigned'].index
# Delete these rows index from the Data Frame
df.drop(indexNames, inplace = True)

In [7]:
# Replacing the Neighbourhood = "Not Assigned" to Borough values
df.loc[(df.Neighbourhood == 'Not assigned'), 'Neighbourhood'] = df['Borough']

In [8]:
# Grouping different neighbourhood based on the postcode.
df = df.groupby('Postcode').agg({'Borough':'first', 
                                    'Neighbourhood': ','.join}).reset_index()

In [9]:
df.to_csv('C:/Users/ragha/Desktop/Projects/PostalCode_Canada.csv', index = False)



In [10]:
print(df.shape)


(103, 3)


In [11]:
####################### Part Two ###########################################

In [12]:
# Import Geospatial csv file
GeospatialData = pd.read_csv('C:/Users/ragha/Downloads/Geospatial_Coordinates.csv')
#print(GeospatialData)

In [13]:
Merger_Data = pd.merge(left = df, right = GeospatialData, left_on = 'Postcode', right_on = 'Postal Code')    

In [14]:
print(Merger_Data[['Postcode','Borough', 'Neighbourhood', 'Latitude', 'Longitude']])

    Postcode           Borough  \
0        M1B       Scarborough   
1        M1C       Scarborough   
2        M1E       Scarborough   
3        M1G       Scarborough   
4        M1H       Scarborough   
5        M1J       Scarborough   
6        M1K       Scarborough   
7        M1L       Scarborough   
8        M1M       Scarborough   
9        M1N       Scarborough   
10       M1P       Scarborough   
11       M1R       Scarborough   
12       M1S       Scarborough   
13       M1T       Scarborough   
14       M1V       Scarborough   
15       M1W       Scarborough   
16       M1X       Scarborough   
17       M2H        North York   
18       M2J        North York   
19       M2K        North York   
20       M2L        North York   
21       M2M        North York   
22       M2N        North York   
23       M2P        North York   
24       M2R        North York   
25       M3A        North York   
26       M3B        North York   
27       M3C        North York   
28       M3H  

In [15]:
#Final data
df.to_csv('C:/Users/ragha/Desktop/Projects/FinalData.csv', index = False)

In [20]:
###############   Explore Data ###############

### Download all the dependencies ######
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import folium
import geocoder
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
from geopy.geocoders import Nominatim

In [26]:
# Assigning Merger Data which has longitudes and latitudes to FinalData
FinalData = Merger_Data


In [27]:
FinalData.head(5)

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [30]:
# Display the Columns
FinalData.columns

Index(['Postcode', 'Borough', 'Neighbourhood', 'Postal Code', 'Latitude',
       'Longitude'],
      dtype='object')

In [36]:
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent ='Canada_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [47]:
print('The data frame has {} boroughs and {} Neighbourhood.'.format(len(FinalData['Borough'].unique()), FinalData.shape[0]))

The data frame has 11 boroughs and 103 Neighbourhood.


# Creating a Map of Toronto

In [57]:
map_toronto = folium.Map(location = [latitude, longitude])

for lat, lng, borough, neighbourhood in zip(FinalData['Latitude'], FinalData['Longitude'], FinalData['Borough'], FinalData['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker( 
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_toronto)

map_toronto 
    

In [58]:
LIMIT = 100
CLIENT_ID = 'DGAZKXRPMEK22CQC4GONQOFZJD4K2ZEJNKHAPN0WAANLVFNP'
CLIENT_SECRET = 'TR2UYMFXDHNHBYRSCN3T4IPLLPKBTCD2O2TVK1CS1PXGBHTZ'
VERSION = '20190921'

In [65]:
####### Explore Neighbourhoods in Toronto Canada
def getNearbyVenues(names, latitudes, longitudes, radius = 500):
    
    venues_list = []
    for name, lat,lng in zip(names, latitudes, longitudes):
        print(name)
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
    
        results = requests.get(url).json()["response"]["groups"][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

toronto_venues = getNearbyVenues(names=FinalData['Neighbourhood'],
                                   latitudes=FinalData['Latitude'],
                                   longitudes=FinalData['Longitude'],
                                  )

Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beaches West,Indi

In [66]:

toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"Guildwood,Morningside,West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


In [67]:
toronto_venues.shape

(2270, 7)

In [68]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Agincourt,5,5,5,5,5,5
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",3,3,3,3,3,3
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",10,10,10,10,10,10
"Alderwood,Long Branch",11,11,11,11,11,11
"Bathurst Manor,Downsview North,Wilson Heights",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",22,22,22,22,22,22
Berczy Park,56,56,56,56,56,56


In [69]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 273 uniques categories.


In [70]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [71]:

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"Adelaide,King,Richmond",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.020000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.0
1,Agincourt,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
4,"Alderwood,Long Branch",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
5,"Bathurst Manor,Downsview North,Wilson Heights",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.0
6,Bayview Village,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
7,"Bedford Park,Lawrence Manor East",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
8,Berczy Park,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
9,"Birch Cliff,Cliffside West",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


In [72]:

num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
         venue  freq
0  Coffee Shop  0.08
1         Café  0.05
2   Steakhouse  0.04
3          Bar  0.04
4   Restaurant  0.03


----Agincourt----
                venue  freq
0              Lounge   0.2
1      Breakfast Spot   0.2
2        Skating Rink   0.2
3      Sandwich Place   0.2
4  Chinese Restaurant   0.2


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                venue  freq
0          Playground  0.33
1         Coffee Shop  0.33
2                Park  0.33
3         Yoga Studio  0.00
4  Mexican Restaurant  0.00


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
                 venue  freq
0        Grocery Store   0.2
1          Pizza Place   0.1
2  Fried Chicken Joint   0.1
3          Coffee Shop   0.1
4       Sandwich Place   0.1


----Alderwood,Long Branch----
          venue  freq
0   Pizza Place  0.18
1           Pub  0.09
2      Pharmacy  0.09
3           Gym 

           venue  freq
0  Grocery Store  0.33
1  Shopping Mall  0.17
2           Park  0.17
3           Bank  0.17
4          Hotel  0.17


----Downsview,North Park,Upwood Park----
                        venue  freq
0            Basketball Court  0.25
1                      Bakery  0.25
2  Construction & Landscaping  0.25
3                        Park  0.25
4                 Yoga Studio  0.00


----East Birchmount Park,Ionview,Kennedy Park----
               venue  freq
0     Discount Store  0.33
1        Bus Station  0.17
2   Department Store  0.17
3  Convenience Store  0.17
4        Coffee Shop  0.17


----East Toronto----
                venue  freq
0                Park  0.33
1         Coffee Shop  0.33
2   Convenience Store  0.33
3         Yoga Studio  0.00
4  Mexican Restaurant  0.00


----Emery,Humberlea----
                             venue  freq
0       Construction & Landscaping   0.5
1                   Baseball Field   0.5
2                      Yoga Studio   0.0
3       

                venue  freq
0                Café  0.08
1    Sushi Restaurant  0.06
2         Pizza Place  0.06
3  Italian Restaurant  0.06
4         Coffee Shop  0.06


----Ryerson,Garden District----
                       venue  freq
0             Clothing Store  0.08
1                Coffee Shop  0.08
2                       Café  0.03
3             Cosmetics Shop  0.03
4  Middle Eastern Restaurant  0.03


----Scarborough Village----
                        venue  freq
0                  Playground   0.5
1                  Smoke Shop   0.5
2                 Yoga Studio   0.0
3               Metro Station   0.0
4  Modern European Restaurant   0.0


----St. James Town----
                venue  freq
0         Coffee Shop  0.07
1                Café  0.06
2               Hotel  0.05
3          Restaurant  0.05
4  Italian Restaurant  0.04


----Stn A PO Boxes 25 The Esplanade----
                venue  freq
0         Coffee Shop  0.11
1          Restaurant  0.04
2                Café  

In [73]:

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [74]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Steakhouse,Bar,Hotel,Restaurant,Cosmetics Shop,Burger Joint,American Restaurant,Thai Restaurant
1,Agincourt,Lounge,Skating Rink,Sandwich Place,Breakfast Spot,Chinese Restaurant,Electronics Store,Eastern European Restaurant,Empanada Restaurant,Dumpling Restaurant,Dessert Shop
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Coffee Shop,Playground,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Fried Chicken Joint,Beer Store,Fast Food Restaurant,Sandwich Place,Liquor Store,Coffee Shop,Pizza Place,Pharmacy,Comfort Food Restaurant
4,"Alderwood,Long Branch",Pizza Place,Gym,Athletics & Sports,Pub,Sandwich Place,Pool,Dance Studio,Pharmacy,Skating Rink,Coffee Shop


In [76]:

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 3, 0, 3, 3, 3, 3, 3, 3, 3])

In [85]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)



# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
FinalData = FinalData.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(FinalData['Latitude'], FinalData['Longitude'], FinalData['Neighbourhood'], FinalData['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

TypeError: list indices must be integers or slices, not float

In [86]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(FinalData['Latitude'], FinalData['Longitude'], FinalData['Neighbourhood'], FinalData['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [89]:
FinalData.loc[FinalData['Cluster Labels'] == 0, FinalData.columns[[1] + list(range(5, FinalData.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,-79.284577,0.0,Park,Coffee Shop,Playground,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
23,North York,-79.400049,0.0,Park,Bank,Convenience Store,Women's Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
40,East York,-79.338106,0.0,Convenience Store,Park,Coffee Shop,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
44,Central Toronto,-79.388790,0.0,Park,Swim School,Bus Line,Dim Sum Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dessert Shop,Donut Shop
50,Downtown Toronto,-79.377529,0.0,Park,Playground,Building,Trail,Eastern European Restaurant,Electronics Store,Dumpling Restaurant,Drugstore,Donut Shop,Deli / Bodega
64,Central Toronto,-79.411307,0.0,Trail,Park,Bus Line,Sushi Restaurant,Jewelry Store,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant
72,North York,-79.445073,0.0,Park,Pizza Place,Pub,Japanese Restaurant,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
90,Etobicoke,-79.506944,0.0,Pool,Smoke Shop,River,Park,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Empanada Restaurant
98,York,-79.518188,0.0,Convenience Store,Park,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Women's Store
100,Etobicoke,-79.554724,0.0,Park,Pizza Place,Bus Line,Mobile Phone Shop,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Department Store,Doner Restaurant


In [90]:
FinalData.loc[FinalData['Cluster Labels'] == 2, FinalData.columns[[1] + list(range(5, FinalData.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,-79.194353,2.0,Fast Food Restaurant,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Wings Joint
25,North York,-79.329656,2.0,Park,Food & Drink Shop,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
30,North York,-79.464763,2.0,Park,Airport,Construction & Landscaping,Women's Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
74,York,-79.453512,2.0,Park,Women's Store,Market,Fast Food Restaurant,Comfort Food Restaurant,Comic Shop,Event Space,Ethiopian Restaurant,Empanada Restaurant,Colombian Restaurant
79,North York,-79.490074,2.0,Basketball Court,Park,Bakery,Construction & Landscaping,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore


In [91]:
FinalData.loc[FinalData['Cluster Labels'] == 3, FinalData.columns[[1] + list(range(5, FinalData.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,-79.188711,3.0,Rental Car Location,Breakfast Spot,Intersection,Mexican Restaurant,Medical Center,Pizza Place,Electronics Store,Dumpling Restaurant,Drugstore,Donut Shop
3,Scarborough,-79.216917,3.0,Coffee Shop,Soccer Field,Korean Restaurant,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
4,Scarborough,-79.239476,3.0,Hakka Restaurant,Caribbean Restaurant,Bank,Bakery,Fried Chicken Joint,Thai Restaurant,Athletics & Sports,Cosmetics Shop,Coworking Space,Event Space
5,Scarborough,-79.239476,3.0,Smoke Shop,Playground,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Dumpling Restaurant
6,Scarborough,-79.262029,3.0,Discount Store,Department Store,Bus Station,Coffee Shop,Convenience Store,Drugstore,Diner,Dog Run,Doner Restaurant,Donut Shop
7,Scarborough,-79.284577,3.0,Bakery,Bus Line,Soccer Field,Park,Fast Food Restaurant,Bus Station,Intersection,Metro Station,Doner Restaurant,Dog Run
8,Scarborough,-79.239476,3.0,Motel,American Restaurant,Women's Store,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
9,Scarborough,-79.264848,3.0,General Entertainment,College Stadium,Café,Skating Rink,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
10,Scarborough,-79.273304,3.0,Indian Restaurant,Pet Store,Vietnamese Restaurant,Light Rail Station,Latin American Restaurant,Chinese Restaurant,Furniture / Home Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant
11,Scarborough,-79.295849,3.0,Auto Garage,Middle Eastern Restaurant,Shopping Mall,Breakfast Spot,Sandwich Place,Smoke Shop,Accessories Store,Concert Hall,Diner,Ethiopian Restaurant


In [92]:
FinalData.loc[FinalData['Cluster Labels'] == 4, FinalData.columns[[1] + list(range(5, FinalData.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,North York,-79.495697,4.0,Home Service,Food Truck,Baseball Field,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Women's Store
91,Etobicoke,-79.498509,4.0,Construction & Landscaping,Pool,Baseball Field,Women's Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
97,North York,-79.532242,4.0,Baseball Field,Construction & Landscaping,Women's Store,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant


In [93]:
FinalData.loc[FinalData['Cluster Labels'] == 5, FinalData.columns[[1] + list(range(5, FinalData.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
